In [ ]:
pip install geopandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from ipywidgets import interactive, IntSlider, Layout, HBox, VBox

us_states = gpd.read_file('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')

us_states['geometry'] = us_states['geometry'].buffer(0)

us_states = us_states.to_crs('EPSG:3395')

us_states['centroid'] = us_states['geometry'].centroid

connections = pd.read_csv('/Users/hazemabo-donia/Desktop/Flow/SummerWork/test2.csv')

connection_dict = {(row['state1'], row['state2'], row['Year']): row['magnitude'] for _, row in connections.iterrows()}

years = connections['Year'].unique()

year_slider = IntSlider(min=min(years), max=max(years), step=1, value=min(years), 
                        description='Year:', 
                        layout=Layout(width='auto', height='auto'))

def update_plot(year):
    plt.clf()

    fig, ax = plt.subplots(1, 1)

    us_states.boundary.plot(ax=ax, linewidth=1, color='black')

    for x, y in zip(us_states['centroid'].x, us_states['centroid'].y):
        plt.plot(x, y, 'bo', markersize=3)

    magnitudes = [v for (s1, s2, yr), v in connection_dict.items() if yr == year]
    if magnitudes:
        min_magnitude, max_magnitude = min(magnitudes), max(magnitudes)
        norm = mcolors.Normalize(min_magnitude, max_magnitude)
        cmap = cm.get_cmap('viridis')

        for idx, state in us_states.iterrows():
            neighbors = us_states[us_states.geometry.touches(state['geometry'])].index.tolist()
            for neighbor in neighbors:
                state_name = state['name']
                neighbor_name = us_states.loc[neighbor, 'name']
                pair = ((state_name, neighbor_name, year) if (state_name, neighbor_name, year) in connection_dict else
                        (neighbor_name, state_name, year))

                if pair in connection_dict:
                    x1, y1 = us_states.loc[idx, 'centroid'].x, us_states.loc[idx, 'centroid'].y
                    x2, y2 = us_states.loc[neighbor, 'centroid'].x, us_states.loc[neighbor, 'centroid'].y
                    ax.plot([x1, x2], [y1, y2], 'k-', color=cmap(norm(connection_dict[pair])), linewidth=.75)

        ax.set_xlim([-14500000, -7000000])
        ax.set_ylim([2500000, 7000000])

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        fig.colorbar(sm, ax=ax, orientation='vertical', label='Energy Transfer')

        plt.show()

interactive_plot = interactive(update_plot, year=year_slider)

VBox([HBox([interactive_plot])])